## Midgap model for surface recombination

In this notebook, we want to calculate the surface recombination rate for silicon. To do this we will use a modified Shockley-Read-Hall formula.

$$ U_{s} = \int_{E_{v}}^{E_{c}} \frac{ \left( n_{s}p_{s} - n_{i}^{2} \right)D_{it}dE } {\frac{1}{\sigma_{p}}\left( n_{s} + n_{1} \right) + \frac{1}{\sigma_{n}}\left( p_{s} + p_{1} \right)}  $$

where $n_{s}, p_{s}$ are the surface concentrations of electrons and holes, $n_{i} = 9.65\times 10^{9}$~charges/cm$^{3}$ is the intrinsic charge number concentration, $D_{it}$ is the density of interface defects, $\sigma_{n}, \sigma_{p}$ are the capture cross sections.

In general the surface recombination should be dominated by the midgap state. So a simpler model might be to assume that there is only one trap state at the midgap, i.e. $D_{it}(E) = D_{it}\delta \left( E_{v} + \frac{E_{gap}}{2} \right)$. Then the surface recombination will be given by

$$ U_{s} = \frac{ \left( n_{s}p_{s} - n_{i}^{2} \right)D_{it}} {\frac{1}{\sigma_{p}}\left( n_{s} + n_{1} \right) + \frac{1}{\sigma_{n}}\left( p_{s} + p_{1} \right)}  $$

Below, we import the relevant python modules, and define some useful constants.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.integrate import simps
%matplotlib inline


# Physical constants
k = 1.38e-23                         # Boltzmann constant (m^2 kg)/(s^2 K)
T = 300                              # Room Temperature (K)
q = 1.6e-19                          # fundamental charge (C)
kT = k*T                             # Thermal energy (J)
beta = 1/kT                          # Inverse thermal energy (1/J)
Vt = kT/q                            # Thermal Voltage (V)
ni = 9.65e9                          # Intrinsic charge concentration of silicon (cm^-3)
eps0 = 8.85e-12                      # Permitivity of free space (F/m)
eps_si = 11.7                        # Dielectric constant for silicon (dimensionless)
eps_ox = 3.8
Li = ((kT*eps_si*eps0)/(2*q*1e6*ni))**2  # Intrinsic diffusion length silicon (m)

Next, define functions to calculate the quasi-Fermi levels.

We want to calculate the quasi-Fermi levels as a function of both minority carrier injection $\Delta n$ and the potential bias induced in the device $V$. By comparing results, we can validate that the functions are working correctly.

To calculate the bias $V$, we write (definitions taken from ABerle 1992)

$$ V = \phi_{p} - \phi_{n} $$

Far from the surface of the material, in the bulk, the total charge density is zero. We can write this as 

$$ p - n + N_{D} - N_{A} = 0 $$

$$ n_{i}\exp\left( \beta \phi_{p} \right) - n_{i}\exp\left( -\beta \phi_{n} \right)  + N_{D} - N_{A} = 0 $$

Combining these two equations, we can calculate the quasi-Fermi levels



In [5]:
# We have a function that tells us the material properties given the number of electrons/holes in an excited state

def calc_phi1(delta_n, material, N):
    # delta n:    minority carrier injection
    # material:   'ntype' or 'ptype'
    # N:          doping level
    if material == 'ntype':
        n0 = N
        p0 = ni**2/N
    elif material == 'ptype':
        n0 = ni**2/N
        p0 = N
    # electron quantities
    n = n0 + delta_n
    phi_n = -Vt*np.log(n/ni)
    # hole quantities
    p = p0 + delta_n
    phi_p = Vt*np.log(p/ni)
    # photovoltage
    V = phi_p - phi_n
    return n0, p0, n, p, phi_n, phi_p, V, delta_n, material, N

delta_n = 1.0e15
material = 'ptype'
N = 1.0e16
n0, p0, n, p, phi_n, phi_p, V, delta_n, material, N = calc_phi1(delta_n, material, N)

# Function to look at the properties of the material given photoinduced voltage V

def calc_phi2(V, material, N):
    # V:          photovoltage
    # material:   'ntype' or 'ptype'
    # N:          doping level
    if material == 'ntype':
        n0 = N
        p0 = ni**2/N
        def f(phi_n): # Charge neutrality condition
            return np.exp((phi_n + V)/Vt) - np.exp(-phi_n/Vt) + N/ni
        def fprime(phi_n): # Jacobian for charge neutrality
            return (1/Vt)*np.exp((phi_n + V)/Vt) + (1/Vt)*np.exp(-phi_n/Vt)
        phi_n = fsolve(f, 2, fprime = fprime)
    elif material == 'ptype':
        p0 = N
        n0 = ni**2/N
        def f(phi_n): # Charge neutrality condition
            return -np.exp((phi_n + V)/Vt) + np.exp(-phi_n/Vt) + N/ni
        def fprime(phi_n): # Jacobian for charge neutrality
            return (1/Vt)*np.exp((phi_n + V)/Vt) + beta*np.exp(-phi_n/Vt)
    [phi_n] = fsolve(f, 2., fprime = fprime)
    phi_p = phi_n + V
    n = ni*np.exp(-phi_n/Vt)
    p = ni*np.exp(phi_p/Vt)
    delta_n = (p - p0)/2. + (n - n0)/2.
    return n0, p0, n, p, phi_n, phi_p, V, delta_n, material, N

# Test to make sure that the two results are consistent with each other
n02, p02, n2, p2, phi_n2, phi_p2, V2, delta_n2, material, N2 = calc_phi2(V, material, N)
print(n0/n02, p0/p02, n/n2, p/p2, phi_n/phi_n2, phi_p/phi_p2, V/V2, delta_n/delta_n2, N/N2)


1.0 1.0 1.0000000000007743 0.9999999999992247 1.000000000000067 0.9999999999999445 1.0 0.9999999999961232 1.0


For given material parameters we want to be able to calculate the surface potential. In order to do this

$$ Q_{Si} = -(2*q*ni*eps0*eps_si/Vt)**(0.5)*F$$

where

$$ F = \exp\left( \frac{\phi_{p} - \psi_{s}}{V_{t}} \right) - \exp(\phi_{p}/V_{t}) + \exp((\psi_{s} - \phi_{n})/V_{t}) - \exp(-\phi_{n}/V_{t}) - \psi_{s}N/(n_{i}V_{t}) $$

as psi gets more negative, the F becomes

$$ 0 +  $$

In [81]:
# Fixed charge density (Coulomb/m^2)
def calc_Qf(Nf):
    return q*Nf

def calc_Qfprime():
    return 0.0

def calc_Qsi(psi_s, n0, p0, n, p, phi_n, phi_p, V, delta_n, material, N):
    # Equation 8a
    #n0, p0, n, p, phi_n, phi_p, V, delta_n, material, N = phi_out
    if material == 'ntype':
        F = np.exp((phi_p + psi_s)/Vt) - np.exp(phi_p/Vt) - np.exp((-psi_s - phi_n)/Vt) + np.exp(-phi_n/Vt) - psi_s*N/(ni*Vt)
    elif material == 'ptype':
        #F = np.exp((phi_p + psi)/Vt) - np.exp(phi_p/Vt) - np.exp((-psi - phi_n)/Vt) + np.exp(-phi_n/Vt) + psi*N/(ni*Vt)
        F = 2.
    const = (2.*q*100**3*kT*ni*eps0*eps_si)**0.5
    return -const*F

def calc_Qit(psi_s, data):
    phi_out, trap
    
    
        
def calc_Qtot(psi_s, *data):
    Vg = 1.0
    dox = 100.0e-9
    Nf, n0, p0, n, p, phi_n, phi_p, V, delta_n, material, N = data
    #n0, p0, n, p, phi_n, phi_p, V, delta_n, material, N = phi_out
    Qf = calc_Qf(Nf)
    Qsi = calc_Qsi(psi_s, n0, p0, n, p, phi_n, phi_p, V, delta_n, material, N)
    return Qsi


phi_out = calc_phi2(0.3, 'ntype', 1.0e15)
n0, p0, n, p, phi_n, phi_p, V, delta_n, material, N = phi_out

#
# Create a plot showing different contributions for silicon charge
#

nsamp = 100
Nf = 1.0e12
data = (Nf, n0, p0, n, p, phi_n, phi_p, V, delta_n, "ntype", N)

[psi_s] = fsolve(calc_Qtot, 0.5, args = data)
print(psi_s)

#psi = np.linspace(0, 0.4, 100)
#Qtot = calc_Qtot(psi)

#plt.plot(psi, Qtot)

0.3641518076520738


In [51]:
print(*data)

1000000000000.0 1000000000000000.0 93122.5 1000010100632753.4 10100632753.098541 -0.2988190609801738 0.0011809390198261882 0.3 10100586191.98677 ntype 1000000000000000.0
